# Day 2015_22: Wizard Simulator 20XX

In [ ]:
year = 2015
day  = 22

In [ ]:
from local_settings import load_input
content = load_input(year, day)
print(f"[{content[:100]}...]")

# Part 1

In [ ]:
# definitions for first part of problem solution
def parseBoss(s):
    result = dict()
    for line in s.splitlines():
        prop, val = line.split(":")
        result[prop.strip()] = int(val.strip())
    return result

class InvalidCast(RuntimeError):
    def __init__(self, msg):
        super().__init__(f"Invalid cast: {msg}")

class BossDied(Exception):
    def __init__(self):
        super().__init__("Boss died")

class PlayerDied(Exception):
    def __init__(self):
        super().__init__("Player died")

def checkMana(player, boss):
    if player["Mana"] < 0:
        raise InvalidCast("Out of mana")

def isActive(character, effect):
    return character.get(effect, 0) > 0

def checkEffect(character, effect):
    if isActive(character, effect):
        raise InvalidCast("Already in effect:" + effect)

def checkHP(player, boss):
    if boss["Hit Points"] <= 0:
        raise BossDied()
    if player["Hit Points"] <= 0:
        raise PlayerDied()

def changeHP(player, boss, playerDelta, bossDelta):
    player["Hit Points"] += playerDelta
    boss["Hit Points"] += bossDelta
    checkHP(player, boss)

def changeMana(player, boss, deltaMana):
    player["Mana"] += deltaMana
    checkMana(player, boss)

def applyEffect(character, effect, time):
    checkEffect(character, effect)
    character[effect] = time

def magicMissile(player, boss, mana):
    changeMana(player, boss, -mana)
    changeHP(player, boss, 0, -4)

def drain(player, boss, mana):
    changeMana(player, boss, -mana)
    changeHP(player, boss, 2, -2)

def shield(player, boss, mana):
    applyEffect(player, "Shield", 6)
    changeMana(player, boss, -mana)

def poison(player, boss, mana):
    applyEffect(boss, "Poison", 6)
    changeMana(player, boss, -mana)

def recharge(player, boss, mana):
    applyEffect(player, "Recharge", 5)
    changeMana(player, boss, -mana)

spells = [(53, magicMissile, "Magic Missile"),
          (73, drain, "Drain"),
          (113, shield, "Shield"),
          (173, poison, "Poison"),
          (229, recharge, "Recharge")]

def tickDown(character, effect):
    if isActive(character, effect):
        character[effect] -= 1
        if not isActive(character, effect):
            del character[effect]

def tickShield(player, boss):
    player["Armor"] = 7 * int(isActive(player, "Shield"))
    tickDown(player, "Shield")

def tickPoison(player, boss):
    changeHP(player, boss, 0, -3 * int(isActive(boss, "Poison")))
    tickDown(boss, "Poison")

def tickRecharge(player, boss):
    changeMana(player, boss, 101 * int(isActive(player, "Recharge")))
    tickDown(player, "Recharge")

def tickEffects(player, boss):
    tickShield(player, boss)
    tickPoison(player, boss)
    tickRecharge(player, boss)

def attack(player, boss):
    attackPower = max(boss["Damage"] - player["Armor"], 1)
    changeHP(player, boss, -attackPower, 0)
        
from heapq import heapify, heappop, heappush

def fight(player, boss, hardMode=False):
    manaUsage = 0
    count = 0
    theHeap = [(manaUsage, count, player, boss, [])]
    bestCost = 10**9
    bestSpells = []
    heapify(theHeap)
    while theHeap:
        manaUsage, _, player0, boss0, spellList = heappop(theHeap)
        if manaUsage >= bestCost:
            continue
        # print(manaUsage, player0, boss0, spellList)
        for spellcost, spell, spellName in spells:
            player, boss = dict(player0), dict(boss0)
            sCost = 0
            try:
                if hardMode:
                    changeHP(player, boss, -1, 0)
                tickEffects(player, boss)
                sCost += spellcost
                spell(player, boss, spellcost)
                tickEffects(player, boss)
                attack(player, boss)
                # print(f"  {spellName}: {player} {boss}")
            except PlayerDied:
                # print(f"  {spellName}: Player died! {player} {boss}")
                continue
            except InvalidCast:
                # print(f"  {spellName}: Invalid cast! {player} {boss}")
                continue
            except BossDied:
                # print(f"  {spellName}: Boss died! {player} {boss}")
                if sCost + manaUsage < bestCost:
                    bestCost = sCost + manaUsage
                    bestSpells = spellList + [spellName]
            count += 1
            heappush(theHeap, (sCost + manaUsage, count, player, boss, spellList + [spellName]))
    return bestCost, bestSpells

## Examples:
```
-- Player turn --
- Player has 10 hit points, 0 armor, 250 mana
- Boss has 13 hit points
Player casts Poison.

-- Boss turn --
- Player has 10 hit points, 0 armor, 77 mana
- Boss has 13 hit points
Poison deals 3 damage; its timer is now 5.
Boss attacks for 8 damage.

-- Player turn --
- Player has 2 hit points, 0 armor, 77 mana
- Boss has 10 hit points
Poison deals 3 damage; its timer is now 4.
Player casts Magic Missile, dealing 4 damage.

-- Boss turn --
- Player has 2 hit points, 0 armor, 24 mana
- Boss has 3 hit points
Poison deals 3 damage. This kills the boss, and the player wins.
```

In [ ]:
# testing the examples
player = {"Hit Points": 10, "Armor": 0, "Mana": 250}
boss = {"Hit Points": 13, "Damage": 8}
mana, sequence = fight(player, boss)
print(f"Mana spent: {mana} [{', '.join(sequence)}]")

In [ ]:
player = {"Hit Points": 10, "Armor": 0, "Mana": 250}
boss = {"Hit Points": 14, "Damage": 8}
mana, sequence = fight(player, boss)
print(f"Mana spent: {mana} [{', '.join(sequence)}]")

In [ ]:
# finding the solution
def new_func():
    boss = parseBoss(content)
    player = {"Mana": 500, "Hit Points": 50, "Armor": 0}
    mana, sequence = fight(player, boss)
    print(f"Mana spent: {mana} [{', '.join(sequence)}]")

new_func()

# Part 2

In [ ]:
# definitions for second part of a problem solution

## Examples:
```
```

In [ ]:
# testing the examples

In [ ]:
# finding the solution
boss = parseBoss(content)
player = {"Mana": 500, "Hit Points": 50, "Armor": 0}
mana, sequence = fight(player, boss, hardMode=True)
print(f"Mana spent: {mana} [{', '.join(sequence)}]")